### 사전 학습된 CNN 모형 활용 예측.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings           
import random
import cv2
import re
import os
from PIL import Image
#from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets.cifar10 import load_data                           
from tensorflow.keras.applications import mobilenet   # 비교적 작은 크기의 모형.
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.imagenet_utils import decode_predictions
warnings.filterwarnings('ignore')                    # Turn the warnings off.
%matplotlib inline

In [2]:
# # GPU를 검출해서 메모리 확장을 허락할 필요가 있다.
# my_gpus = tf.config.experimental.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(my_gpus[0],True)

#### 1. 데이터 불러오기:

자세한 설명은 [이곳](https://www.cs.toronto.edu/~kriz/cifar.html)을 참고한다.

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
# 데이터 가져오기.
(X_train, y_train), (X_test, y_test) = load_data()
n_train_size = X_train.shape[0]
n_test_size = X_test.shape[0]

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d958be074577803d12ecdefd02955f39262c83c16fe9348329d7fe0b5c001ce so we will re-download the data.
170498071/170498071 [==============================] - 73s 0us/step


#### 2. 데이터 준비:

In [5]:
# 주의: Scaling 해주지 않는다.
# 아래 preprocess_input을 적용하기 위해서!

In [6]:
# 4 차원 배열로 reshaping 해준다.
X_train = X_train.reshape(-1,32,32,3)
X_test = X_test.reshape(-1,32,32,3)

#### 3. 사전 학습된 MobileNet 모델을 가져와서 예측에 활용해 본다:

In [7]:
# MobileNet을 가져와 본다.
# 입력 크기는 128, 160, 192, 224 가 가능하다.
my_mnet = mobilenet.MobileNet(include_top=True, weights='imagenet', input_shape=(128,128,3))      

17225924/17225924 [==============================] - 1s 0us/step


In [8]:
# 가져온 모델의 summary (내부 구조) 출력.
# Y의 유형은 1000 가지이다!
#my_mnet.summary()

In [9]:
# 구글 드라이브 마운트.
# 절차를 따라서 한다.
# drive.mount('/content/drive')
# 데이터 파일이 이미 "Colab Notebooks"에 올려져 있어야 한다.
# os.chdir("/content/drive/My Drive/Colab Notebooks")

In [10]:
# 1000개의 ImageNet 레이블을 읽어와서 본다.
with open(r"../data/ImageNet_labels.txt","r") as f:
    res = f.readlines()
my_imagenet_labels=[re.sub("[0-9]|\n","",x ).split(',')[0].strip() for x in res]
#print("Length : ", len(my_imagenet_labels))
#print(my_imagenet_labels)

FileNotFoundError: [Errno 2] No such file or directory: '../data/ImageNet_labels.txt'

In [ ]:
# BGR 또는 RGB channel 순서의 차이와 무관하게 다음 리사이징이 가능하다.
i_image = 1
X_resized = cv2.resize(X_train[i_image,:,:,:],(128,128))                # 방법 #1.
#X_resized = tf.image.resize(X_train[i_image,:,:,:],(128,128)).numpy()    # 방법 #2.

In [ ]:
# 리사이징 전 후 비교.
fig, axes = plt.subplots(1, 2)
axes[0].imshow(X_train[i_image])
axes[0].axis('off')
axes[1].imshow(X_resized)
axes[1].axis('off')
plt.show()

In [ ]:
# 3차원 X_resized를 4차원으로 변환하고 예측한다.
my_pred = my_mnet(mobilenet.preprocess_input(np.expand_dims(X_resized,axis=0)))        

In [ ]:
# 예측 결과에 ImageNet 레이블을 붙여서 해석해 준다.
# 불필요한 튜플의 첫 번째 원소를 제거하고 정리해서 보여준다.
res = [(a_name, a_prob) for _, a_name, a_prob in decode_predictions(my_pred.numpy(),top=5)[0]]    

In [ ]:
pd.Series(dict(res)).plot.bar()
plt.show()

#### 4. 사전 학습된 VGG16 모델을 가져와서 예측에 활용해 본다:

In [ ]:
# VGG16을 가져와 본다.
# 입력 크기는 224 이어야 한다.
my_vgg = vgg16.VGG16(include_top=True, weights='imagenet', input_shape=(224,224,3))  

In [ ]:
# 가져온 모델의 summary (내부 구조) 출력.
# Y의 유형은 1000 가지이다!
# my_vgg.summary()

In [ ]:
# BGR 또는 RGB channel 순서의 차이와 무관하게 다음 리사이징이 가능하다.
i_image = 1
X_resized = cv2.resize(X_train[i_image,:,:,:],(224,224))                # 방법 #1.
#X_resized = tf.image.resize(X_train[i_image,:,:,:],(224,224)).numpy()    # 방법 #2.

In [ ]:
# 리사이징 전 후 비교.
fig, axes = plt.subplots(1, 2)
axes[0].imshow(X_train[i_image])
axes[0].axis('off')
axes[1].imshow(X_resized)
axes[1].axis('off')
plt.show()

In [ ]:
# 3차원 X_resized를 4차원으로 변환하고 예측한다.
my_pred = my_vgg(vgg16.preprocess_input(np.expand_dims(X_resized,axis=0)))

# 예측 결과에 ImageNet 레이블을 붙여서 해석해 준다.
# 불필요한 튜플의 첫 번째 원소를 제거하고 정리해서 보여준다.
res = [(a_name, a_prob) for _, a_name, a_prob in decode_predictions(my_pred.numpy(),top=5)[0]]   

pd.Series(dict(res)).plot.bar()
plt.show()

외부에서 해상도 높은 이미지를 가져와 본다.

In [ ]:
img = Image.open("..\data\panda.jpg")
X = np.array(img)
X_resized = cv2.resize(X,(224,224))        # BGR 또는 RGB channel 순서의 차이와 무관하게 리사이징이 가능하다.

In [ ]:
# 리사이징 전 후 비교.
fig, axes = plt.subplots(1, 2)
axes[0].imshow(X)
axes[0].axis('off')
axes[1].imshow(X_resized)
axes[1].axis('off')
plt.show()

In [ ]:
# 3차원 X_resized를 4차원으로 변환하고 예측한다.
my_pred = my_vgg(vgg16.preprocess_input(np.expand_dims(X_resized,axis=0)))

# 예측 결과에 ImageNet 레이블을 붙여서 해석해 준다.
# 불필요한 튜플의 첫 번째 원소를 제거하고 정리해서 보여준다.
res = [(a_name, a_prob) for _, a_name, a_prob in decode_predictions(my_pred.numpy(),top=5)[0]]   

pd.Series(dict(res)).plot.bar()
plt.show()

In [ ]:
# 레이블을 다시한번 확인해 본다.
my_imagenet_labels[my_pred.numpy().argmax()]